In [1]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import mpl_finance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from datetime import datetime
from openpyxl import load_workbook
import plotly.graph_objects as go
from plotly import subplots

%run _CrawBase.ipynb
%run _BaseInfo.ipynb

df_info=_baseInfo.copy()

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


【craw_stock】 craw_stock stock_number!!!!!!!!! -->8487


In [2]:
def roc_to_gregorian(roc_date_str):
    # 解析字串，假設格式為 "ROC_YEAR/MM/DD"
    roc_year, month, day = roc_date_str.split('/')
    # 轉換為整數
    roc_year = int(roc_year)
    month = int(month)
    day = int(day)
    
    # ROC 年轉西元年
    gregorian_year = roc_year + 1911
    
    # 返回格式化的字串，或是你可以選擇返回 datetime.date 物件
    return pd.to_datetime(f"{gregorian_year:04d}-{month:02d}-{day:02d}")

In [3]:
def save_data_by_date(df, output_dir):
    # 清理无效字符（如 '*'）
    df['日期'] = df['日期'].astype(str).str.replace('*', '', regex=False)

    # 转换为日期格式
    df['日期'] = pd.to_datetime(df['日期'], format='%Y/%m/%d', errors='coerce')

    # 创建存储结果的文件夹
    os.makedirs(output_dir, exist_ok=True)

    # 按日期分组并保存为独立的 Excel 文件
    for date, group in df.groupby('日期'):
        print(date)
        print(roc_to_gregorian(min(end_time_list)))
        if(date>=roc_to_gregorian(min(end_time_list))):
            formatted_date = date.strftime('%Y-%m-%d')  # 格式化日期为文件名
            file_name = f"{output_dir}/{formatted_date}.xlsx"

            # 保存每个分组为单独的 Excel 文件
            group.to_excel(file_name, index=False)

In [4]:
# 計算 RSI 指標
def calculate_rsi(data, window=14):
    delta = data['收盤價'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [5]:
def get_interval(value_to_check, data):
    # 转换数据为浮点数并去除负值和零值
    cleaned_numbers = []
    for num in data:
        try:
            value = float(num)
            if value > 0:
                cleaned_numbers.append(value)
        except ValueError:
            # 忽略无法转换的字符串
            continue
    data= cleaned_numbers  
    #print(value_to_check)
    value_to_check=float(value_to_check)
    
    # 计算均值和标准差
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)  # 使用 ddof=1 计算样本标准差

    confidence_levels = [0.40,0.50,0.65, 0.90, 0.95, 0.99]
    
    best_interval = None
    for level in confidence_levels:
        z_score = stats.norm.ppf(1 - (1 - level) / 2)  # 获取 z 分数
        margin_of_error = z_score * std_dev
        
        lower_bound = mean - margin_of_error
        upper_bound = mean + margin_of_error
        # 检查 value_to_check 是否在当前置信区间内
        if lower_bound <= value_to_check <= upper_bound:
            best_interval = (level, lower_bound, upper_bound)
            break  # 如果找到包含 value_to_check 的置信区间，直接退出
    if best_interval:
        level, lower_bound, upper_bound = best_interval
        if mean <value_to_check:
            interval_type = "正區間"
        else:
            interval_type = "負區間"

        #print(f"数据点 {value_to_check} 落在置信水平 {level * 100}% 的{interval_type}: [{lower_bound:.2f}, {upper_bound:.2f}]")
        return (level,interval_type,lower_bound, upper_bound)
    else:
        if mean <value_to_check:
            interval_type = "正區間"
        else:
            interval_type = "負區間"
        
        return (1,interval_type,value_to_check, value_to_check)
        return None

In [6]:
# 清理函式，只對字串型態欄位進行處理
def clean_column(column):
    if column.dtype == 'object':
        return column.str.replace(',', '').replace('0', np.nan).replace('--', '').apply(pd.to_numeric, errors='coerce')
    return column  # 如果已經是數值型態，直接回傳

### Main 


In [7]:

def data_process(RowData_df_craw_stock):
    # 基本資料清理
    df = RowData_df_craw_stock.copy().drop_duplicates()
    df['日期'] = df['日期'].str.replace("＊", "", regex=False)
    df['日期'] = df['日期'].str.extract(r'(\d{2,3})/(\d{1,2}/\d{1,2})').apply(
        lambda x: f"{int(x[0]) + 1911}/{x[1]}", axis=1
    )
    df['年月日'] = df['日期']

    # 數值欄位轉換
    cols_to_clean = ['成交金額', '收盤價', '開盤價', '最低價', '最高價', '成交股數', '漲跌價差', '成交筆數']
    for col in cols_to_clean:
        if col in df.columns:
            df[col] = clean_column(df[col])

    df['收盤價'] = pd.to_numeric(df['收盤價'], errors='coerce').fillna(method='ffill')
    
    # 前日收盤
    df['前日收盤價'] = df['收盤價'].shift(1)
    
    # ===== 技術指標計算區 =====
    ###########       【價】      ######################
    # MACD 計算　 MACD（指數平滑異同移動平均線）
    ema_12 = df['收盤價'].ewm(span=12, adjust=False).mean()
    ema_26 = df['收盤價'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD-SL'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # MACD 黃金交叉判斷
    df['MACD_golden_cross'] = ((df['MACD'] > df['MACD-SL']) & (df['MACD'].shift(1) <= df['MACD-SL'].shift(1)))

    # KD 計算
    low9 = df['收盤價'].rolling(window=9).min()
    high9 = df['收盤價'].rolling(window=9).max()
    df['%K'] = (df['收盤價'] - low9) / (high9 - low9) * 100
    df['%D'] = df['%K'].rolling(window=3).mean()

    # KD 黃金交叉判斷
    df['KD_golden_cross'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) <= df['%D'].shift(1)))
    
    # MA 計算　MA（移動平均線）與交叉
    df['MA_short'] = df['收盤價'].rolling(window=5).mean()
    df['MA_long'] = df['收盤價'].rolling(window=20).mean()
    df['MA_longlong'] = df['收盤價'].rolling(window=50).mean()
    df['MA_break'] = (df['MA_short'] > df['MA_long']) & (df['MA_short'].shift(3) <= df['MA_long'].shift(3))
    
    # 波動率與變化率
    #df['MA_short_volatility'] = df['MA_short'].rolling(window=10).std()
    #df['MA_long_volatility'] = df['MA_long'].rolling(window=10).std()
    #df['Is_Entangled'] = df['MA_short_volatility'] > df['MA_long_volatility']
    #df['MA_short_change'] = df['MA_short'].diff()
    #df['MA_long_change'] = df['MA_long'].diff()
    
    # 布林帶
    #window = 20
    #num_std_dev = 2
    #df['Bollinger_MA'] = df['收盤價'].rolling(window=window).mean()
    #df['Bollinger_STD'] = df['收盤價'].rolling(window=window).std()
    #df['Bollinger_upper'] = df['Bollinger_MA'] + num_std_dev * df['Bollinger_STD']
    #df['Bollinger_lower'] = df['Bollinger_MA'] - num_std_dev * df['Bollinger_STD']
    #df['Bollinger_breakout'] = df['收盤價'] > df['Bollinger_upper']
    
    # RSI
    df['RSI'] = calculate_rsi(df)
    df['RSI_rebound'] = (df['RSI'] > 30) & (df['RSI'].shift(1) <= 30)
    
    ###########       【量】      ######################
    
   # 價格變動方向加權成交金額
    df['Volume_Price_Change'] = np.where(df['收盤價'].diff() > 0, 1, -1) * df['成交金額']

    # 成交量移動平均與震盪指標
    df['Volume_MA_short'] = df['Volume_Price_Change'].rolling(window=5).mean()
    df['Volume_MA_long'] = df['Volume_Price_Change'].rolling(window=10).mean()
    df['Volume_Oscillator'] = ((df['Volume_MA_short'] - df['Volume_MA_long']) / df['Volume_MA_long']) * 100

    # VPC MACD 系列
    df['VPC_MACD'] = df['Volume_Price_Change'].ewm(span=9, adjust=False).mean() - \
                     df['Volume_Price_Change'].ewm(span=15, adjust=False).mean()
    df['VPC_SIGNAL'] = df['VPC_MACD'].ewm(span=9, adjust=False).mean()
    df['VPC_DIF'] = df['VPC_MACD'] - df['VPC_SIGNAL']

    # 交叉與門檻
   # 動能交叉幅度門檻：改小一些，讓條件不太嚴苛
    threshold = df['VPC_DIF'].rolling(window=10).std() * 0.1
    # volume_threshold 改用穩定的長期平均，避免波動太大
    df['volume_threshold'] = df['VPC_MACD'].ewm(span=12, adjust=False).mean()
    # 成交金額門檻：用絕對成交金額平均（非加權）來代表市場活躍程度
    volume_threshold = df['成交金額'].rolling(window=10).mean()

    # 黃金交叉觸發條件
    df['Volume_Price_Change_break'] = (
        (df['VPC_MACD'] > df['VPC_SIGNAL']) &
        (df['VPC_MACD'] > df['volume_threshold']) &
        (df['VPC_DIF'] > threshold) & 
        (df['成交金額'] > volume_threshold) & # 活躍才觸發
        (df['VPC_MACD'].shift(1) >= df['VPC_SIGNAL'].shift(1)) # 過去交叉持續
    )
    
    # 成交量擴增與變動率
    
    # 價格漲跌方向
    direction = df['收盤價'].diff().apply(lambda x: 1 if x > 0 else -1)
    # 避免除以 0 或 NaN
    safe_volume_ma = df['Volume_MA_short'].replace(0, np.nan)
    
    # 計算短期交易量波動
    
    短交易量_raw = ((df['成交金額'] - safe_volume_ma) / safe_volume_ma) * direction
    
    # 整理並放大比例，用 rolling 滑動加總
    df['短交易量'] = (短交易量_raw
        .replace([np.inf, -np.inf], 0)     # 移除 inf
        .fillna(0)                         # 補 NaN
        .rolling(window=5).sum()          # 滑動加總
        .fillna(0)                         # 再補一次 NaN（可能前5天不足）
        .mul(20)                           # 先乘 20
        .astype(float)                    # 保險轉 float
        .fillna(0)                         # 最後一次保險
        .astype(int)                      # 最終轉 int
    )

    # 「遠交易量」為短期均量與長期均量差異百分比
    df['遠交易量'] = ((df['Volume_MA_short'] - df['Volume_MA_long']) / df['Volume_MA_long']).replace([np.inf, -np.inf], 0).fillna(0) * 100
    df['遠交易量'] = df['遠交易量'].astype(int)

    # 「短增量」：短交易量的強度變化
    df['短增量'] = (abs(df['短交易量']) / df['短交易量'].shift(1).replace(0, np.nan)).pow(0.5)
    df['短增量'] = df['短增量'].fillna(0).rolling(window=3).mean()
    
    # 資料時間轉換與亮點處理
    df['年月日'] = pd.to_datetime(df['年月日'].astype(str).str.replace('*', '', regex=False), format='%Y-%m-%d')
    df, merged_intervals = get_highlight(df)
    ###########################################################################################
    
    # MACD 上升趨勢
    df['macd_golden_crosses_area'] = (
        (df['MACD'] > df['MACD-SL']) &
        (df['MACD'].shift(1) >= df['MACD-SL'].shift(1))
    )
    #同 key_area
    df['VPC_break'] =(
        #(df['MA_short'] > df['MA_long']) &
        #(df['%K'] > df['%D']) &
        #(df['MACD'] > df['MACD-SL']) &  
        #(df['遠交易量'].shift(3) <df['遠交易量'].shift(1)) &  
        #(df['遠交易量'] > -25)& 
        #(df['短交易量'] > -0)
        (df['MACD-SL'] >0)
        & (df['Volume_Price_Change_break'] )
        &  (df['macd_golden_crosses_area'] )
    )
   
    ###########################################################################################
    # 判斷成交量震盪指標是否出現有效突破訊號
    df['VO_Positive'] = df['Volume_Oscillator'] > 5
    df['VO_Positive_Count'] = df['VO_Positive'].rolling(window=3).sum()
    
    # 同 buy_points
    df['buy_points'] =(
        (df['Volume_Oscillator'] > 5) &                              # 當日 VO 大於 5（濾除雜訊）
        (df['Volume_Oscillator'].shift(1) <= 0) &                   # 前一天小於等於 0（突破條件）
        (df['MA_short'] > df['MA_short'].shift(1)) &       # 均線上揚（確認價穩）
        (df['VO_Positive_Count'] >= 2)                              # 近三日至少兩日有效放量
    )
    
    # =====  UI 顯示顏色標記=====
    df['Bar_Color'] = df['收盤價'].diff().apply(lambda x: 'red' if x > 0 else 'green')
    
    # ===== 分類邏輯區 =====
    df['Full_Summary'] = ''
    df = full_technical_analysis(df)
    
    return df

In [8]:
def full_technical_analysis(df):
    """
    統一處理流程：先進行 K棒分析，再執行市場分類與建議
    """
    df = analyze_candlestick(df)
    df = analyze_candlestick_multiday(df)
    df = apply_market_classification(df)
    return df

def analyze_candlestick(df):
    """
    根據 K 棒型態分析，標記常見型態並評估方向：
    - 長紅 / 長黑 K 棒
    - 十字線
    - 上影線長 / 下影線長
    - 多頭吞噬 / 空頭吞噬
    - 錘頭線 / 吊人線 / 流星線
    """
    df = df.copy()
    df['實體長度'] = abs(df['收盤價'] - df['開盤價'])
    df['K棒型態'] = ''

    # 基本形態
    df.loc[(df['收盤價'] > df['開盤價']) & (df['實體長度'] > (df['最高價'] - df['最低價']) * 0.7), 'K棒型態'] = '長紅K棒'
    df.loc[(df['收盤價'] < df['開盤價']) & (df['實體長度'] > (df['最高價'] - df['最低價']) * 0.7), 'K棒型態'] = '長黑K棒'
    df.loc[(df['實體長度'] <= (df['最高價'] - df['最低價']) * 0.1), 'K棒型態'] = '十字線'

    # 上下影線判斷
    df['上影'] = df['最高價'] - df[['收盤價', '開盤價']].max(axis=1)
    df['下影'] = df[['收盤價', '開盤價']].min(axis=1) - df['最低價']
    df.loc[df['上影'] > df['實體長度'], 'K棒型態'] += '|上影線長'
    df.loc[df['下影'] > df['實體長度'], 'K棒型態'] += '|下影線長'

    # 吞噬形態
    df['昨收'] = df['收盤價'].shift(1)
    df['昨開'] = df['開盤價'].shift(1)
    df['昨高'] = df[['昨收', '昨開']].max(axis=1)
    df['昨低'] = df[['昨收', '昨開']].min(axis=1)
    df['今高'] = df[['收盤價', '開盤價']].max(axis=1)
    df['今低'] = df[['收盤價', '開盤價']].min(axis=1)
    df.loc[(df['收盤價'] > df['開盤價']) & (df['昨收'] < df['昨開']) &
           (df['今高'] > df['昨高']) & (df['今低'] < df['昨低']), 'K棒型態'] += '|多頭吞噬'
    df.loc[(df['收盤價'] < df['開盤價']) & (df['昨收'] > df['昨開']) &
           (df['今高'] > df['昨高']) & (df['今低'] < df['昨低']), 'K棒型態'] += '|空頭吞噬'

    # 錘頭 / 吊人 / 流星
    df.loc[
        (df['實體長度'] < (df['最高價'] - df['最低價']) * 0.3) &
        (df['下影'] > df['實體長度'] * 2) &
        (df['上影'] < df['實體長度'] * 0.3), 'K棒型態'
    ] += '|錘頭線或吊人線'

    df.loc[
        (df['實體長度'] < (df['最高價'] - df['最低價']) * 0.3) &
        (df['上影'] > df['實體長度'] * 2) &
        (df['下影'] < df['實體長度'] * 0.3), 'K棒型態'
    ] += '|流星線'

    # K棒方向標註
    df['K棒方向'] = '中性'
    df.loc[df['K棒型態'].str.contains('長紅K棒|下影線長|多頭吞噬|錘頭線'), 'K棒方向'] = '正向'
    df.loc[df['K棒型態'].str.contains('長黑K棒|上影線長|空頭吞噬|流星線|吊人線'), 'K棒方向'] = '負向'
    df.loc[df['K棒型態'].str.contains('十字線'), 'K棒方向'] = '觀望'

    return df

def analyze_candlestick_multiday(df):
    """
    判斷三日 K 棒型態（如晨星、暮星、三白兵、三隻烏鴉）與方向
    """
    df = df.copy()
    df['多日K棒型態'] = ''
    df['多日K棒方向'] = ''

    df['三白兵'] = (
        (df['收盤價'] > df['開盤價']) &
        (df['收盤價'].shift(1) > df['開盤價'].shift(1)) &
        (df['收盤價'].shift(2) > df['開盤價'].shift(2)) &
        (df['收盤價'] > df['收盤價'].shift(1)) &
        (df['收盤價'].shift(1) > df['收盤價'].shift(2))
    )
    df.loc[df['三白兵'], ['多日K棒型態', '多日K棒方向']] = ['|三白兵', '正向']

    df['三隻烏鴉'] = (
        (df['收盤價'] < df['開盤價']) &
        (df['收盤價'].shift(1) < df['開盤價'].shift(1)) &
        (df['收盤價'].shift(2) < df['開盤價'].shift(2)) &
        (df['收盤價'] < df['收盤價'].shift(1)) &
        (df['收盤價'].shift(1) < df['收盤價'].shift(2))
    )
    df.loc[df['三隻烏鴉'], ['多日K棒型態', '多日K棒方向']] = ['|三隻烏鴉', '負向']

    df['晨星'] = (
        (df['收盤價'].shift(2) < df['開盤價'].shift(2)) &
        (abs(df['收盤價'].shift(1) - df['開盤價'].shift(1)) < (df['最高價'].shift(1) - df['最低價'].shift(1)) * 0.1) &
        (df['收盤價'] > df['開盤價']) &
        (df['收盤價'] > (df['收盤價'].shift(2) + df['開盤價'].shift(2)) / 2)
    )
    df.loc[df['晨星'], ['多日K棒型態', '多日K棒方向']] = ['|晨星', '正向']

    df['暮星'] = (
        (df['收盤價'].shift(2) > df['開盤價'].shift(2)) &
        (abs(df['收盤價'].shift(1) - df['開盤價'].shift(1)) < (df['最高價'].shift(1) - df['最低價'].shift(1)) * 0.1) &
        (df['收盤價'] < df['開盤價']) &
        (df['收盤價'] < (df['收盤價'].shift(2) + df['開盤價'].shift(2)) / 2)
    )
    df.loc[df['暮星'], ['多日K棒型態', '多日K棒方向']] = ['|暮星', '負向']

    return df

def apply_market_classification(df):
    import pandas as pd

    df['Market_State'] = ''
    df.loc[(df['收盤價'] < df['開盤價']) & ((df['最高價'] - df['開盤價']) / df['開盤價'] > 0.03), 'Market_State'] += ',衝高回落'
    df.loc[(df['收盤價'] > df['前日收盤價']) & (df['RSI'] < 30), 'Market_State'] += ',低檔翻揚'
    df.loc[(df['收盤價'] > df['MA_long'] * 0.98) & (df['收盤價'] < df['MA_long'] * 1.02) & (df['RSI'].between(45, 55)), 'Market_State'] += ',盤整震盪'
    df.loc[(df['MA_short'] > df['MA_long']) & (df['MA_long'] > df['MA_longlong']), 'Market_State'] += ',多頭排列'
    df.loc[(df['MA_short'] < df['MA_long']) & (df['MA_long'] < df['MA_longlong']), 'Market_State'] += ',空頭排列'
    df.loc[(df['收盤價'] > df['前日收盤價']) & (df['成交金額'] > df['Volume_MA_long'] * 1.5), 'Market_State'] += ',爆量上攻'
    df['Market_State'] = df['Market_State'].str.lstrip(',')

    df['Buy_Signal'] = ''
    df.loc[
        (df['Market_State'].str.contains('多頭排列')) &
        (df['KD_golden_cross']) &
        (df['Market_State'].str.contains('爆量上攻')),
        'Buy_Signal'
    ] = '建議關注買點'

    df['Sell_Signal'] = ''
    df.loc[
        (df['Market_State'].str.contains('空頭排列')) &
        ((df['MACD'] < df['MACD-SL']) & (df['MACD'].shift(1) >= df['MACD-SL'].shift(1))) &
        (df['收盤價'] < df['MA_long']),
        'Sell_Signal'
    ] = '建議留意風險'

    df['Watch_Signal'] = ''
    df.loc[
        (df['Market_State'].str.contains('盤整震盪')) &
        (~df['KD_golden_cross']) &
        (~df['RSI_rebound']) &
        (~((df['MACD'] > df['MACD-SL']) & (df['MACD'].shift(1) <= df['MACD-SL'].shift(1)))),
        'Watch_Signal'
    ] = '觀望為宜'

    df['Reversal_Signal'] = ''
    df['RSI_diff'] = df['RSI'].diff()
    df.loc[
        (df['Market_State'].str.contains('空頭排列')) &
        (df['RSI'] < 30) &
        (df['RSI_diff'] > 5),
        'Reversal_Signal'
    ] = '可能出現反轉訊號'

    df['Action_Advice'] = ''
    df['Advice_Score'] = 0
    df.loc[df['Watch_Signal'] != '', ['Action_Advice', 'Advice_Score']] = ['觀望為宜', 1]
    df.loc[df['Buy_Signal'] != '', ['Action_Advice', 'Advice_Score']] = ['建議關注買點', 2]
    df.loc[df['Reversal_Signal'] != '', ['Action_Advice', 'Advice_Score']] = ['可能出現反轉訊號', 3]
    df.loc[df['Sell_Signal'] != '', ['Action_Advice', 'Advice_Score']] = ['建議留意風險', 4]

    df['Full_Summary'] = (
        '<b>分類：</b>' + df['Market_State'].fillna('') + '<br>' +
        '<b>建議：</b>' + df['Action_Advice'].fillna('') + '<br>' +
        '<b>評分：</b>' + df['Advice_Score'].astype(str) + '<br>' +
        '<b>K棒方向：</b>' + df.get('K棒方向', pd.Series([''] * len(df))) + '<br>' +
        '<b>多日K棒方向：</b>' + df.get('多日K棒方向', pd.Series([''] * len(df))) + '<br>' +
        '<b>K棒續強：</b>' + df.get('K棒續強確認', pd.Series([''] * len(df))) + '<br>' +
        '<b>多日K棒型態：</b>' + df.get('多日K棒型態', pd.Series([''] * len(df)))
    )

    return df


In [9]:
def save_plt_to_html(stock_number,stock_data,fig,text_area):
    try:
        directory = f"Html"    #---本地路徑
        _filename=GetStockInfoByID(stock_number).replace('*', '')
        #print(_filename)
        #檔案路徑設定
        _FilePath = f"{directory}/[{stock_number}]{_filename}.html"  #---本地路徑

        #確認資料夾是否已存在
        if not os.path.exists(directory):
            os.makedirs(directory)

        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++html處理額外新增
        #先存下html在取出，針對head 寫入
        # 需先存下html
        fig.write_html(_FilePath)

        # 读取现有的 HTML 文件
        with open(_FilePath, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

        # html insert 
        head = soup.find('head')
        head.append(BeautifulSoup(text_area, "html.parser"))
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++完成html內容部分

        ## 保存修改后的 HTML 文件 所有的股票都會先存下
        with open(_FilePath, "w", encoding="utf-8") as file:
            file.write(str(soup))

        lastdata=stock_data.iloc[-1]
        
    except Exception as error:
        print(f'Error save_plt_to_html processing  {stock_number}: {error}')

In [10]:
# 找到所有 Upper -> Down 的時間段
def find_intervals(upper, down):
    intervals = []
    for u in upper:
        for d in down:     
            if d > u:
                intervals.append([u, d])
                break
    return intervals

# 合併重維時間段，同時處理重複的標記
def merge_intervals(intervals):
    if not intervals:
        return intervals
    
    # 按每個區間的开始日期排序
    intervals.sort(key=lambda x: x[0])
    merged = []

    for current in intervals:
        if not merged:
            merged.append(current)
        else:
            last = merged[-1]
            if current[0] <= last[1] and current[1] > last[1]:
                # 分割出重複部分，並新增一個新區間
                merged.append([current[0], last[1]])
                merged.append([last[1], current[1]])
            elif current[0] > last[1]:
                merged.append(current)
            else:
                # 更新最後一個區間的結束日期
                last[1] = max(last[1], current[1])

    return merged

In [11]:
def get_highlight(stock_data):
    stock_data['highlight']=False
     
    macd_death_crosses = stock_data[
            (stock_data['MACD'] < stock_data['MACD-SL']) &  # 當前 MACD 線低於信號線
            (stock_data['MACD'].shift(1) >= stock_data['MACD-SL'].shift(1))  # 前一日 MACD 線高於或等於信號線
        ]
    #
    key_area = stock_data[
        (stock_data['MA_short'] > stock_data['MA_long']) &
        (stock_data['%K'] > stock_data['%D']) &
        (stock_data['MACD'] > stock_data['MACD-SL']) &  
        (stock_data['遠交易量'].shift(3) <stock_data['遠交易量'].shift(1)) &  
        (stock_data['遠交易量'] > -25)& 
        (stock_data['短交易量'] > -0)
    ]
    #
    Upper= list(key_area['年月日'])
    Down =list(macd_death_crosses['年月日'])

    Upper.sort()
    Down.sort()
    
    Down.append(pd.Timestamp(datetime.today().date() + timedelta(days=1)))
    intervals = find_intervals(Upper, Down)
    merged_intervals = merge_intervals(intervals)
 
    temp_dates =stock_data['年月日'] 
    
    # 遍历 merged_intervals 进行筛选
    for start, end in merged_intervals:
        mask = (temp_dates >= start) & (temp_dates <= end)
        stock_data.loc[mask, 'highlight'] = True
        stock_data.loc[mask, 'highlight_date'] = start
        stock_data.loc[mask, 'highlight_enddate'] = end

    return stock_data,merged_intervals

In [12]:
def gen_html(stock_number,stock_data):
    
    # 計算 MACD 黃金交叉和死亡交叉
    # 黃金交叉：當前 MACD 線由下而上穿過信號線
    macd_golden_crosses = stock_data[
        (stock_data['MACD'] > stock_data['MACD-SL']) &  # 當前 MACD 線高於信號線
        (stock_data['MACD'].shift(1) <= stock_data['MACD-SL'].shift(1))  # 前一日 MACD 線低於或等於信號線
    ]
 
    # 死亡交叉：當前 MACD 線由上而下穿過信號線
    macd_death_crosses = stock_data[
        (stock_data['MACD'] < stock_data['MACD-SL']) &  # 當前 MACD 線低於信號線
        (stock_data['MACD'].shift(1) >= stock_data['MACD-SL'].shift(1))  # 前一日 MACD 線高於或等於信號線
    ]
    
    stock_data,merged_intervals=get_highlight(stock_data)

    # 計算 RSI 的買入點－找出超賣區回升的點
    # 當 %K 線從 20 以上跌破 20，且 %D 線也在 20 以下時，標記為 RSI 買入點
    rsi_buy_points = stock_data[
        (stock_data['%K'] < 20) &  # 當前 %K 線低於 20
        (stock_data['%K'].shift(1) >= 20) &  # 前一日 %K 線高於或等於 20
        (stock_data['%D'] < 20)  # 當前 %D 線低於 20
    ]

    # 找出超買區回落的點
    # 當 %K 線從 80 以下上升到 80 以上，且 %D 線也在 80 以上時，標記為 RSI 賣出點
    rsi_sell_points = stock_data[
        (stock_data['%K'] > 80) &  # 當前 %K 線高於 80
        (stock_data['%K'].shift(1) <= 80) &  # 前一日 %K 線低於或等於 80
        (stock_data['%D'] > 80)  # 當前 %D 線高於 80
    ]

    # 找出適合買入的點（成交量震盪指標大於 0）
    # 當前成交量震盪指標大於 0，且前一日成交量震盪指標小於或等於 0，標記為買入點
    #buy_points = stock_data[
    #    (stock_data['Volume_Oscillator'] > 0) &  # 當前成交量震盪指標大於 0
    #    (stock_data['Volume_Oscillator'].shift(1) <= 0)  # 前一日成交量震盪指標小於或等於 0
    #]

    # 判斷成交量震盪指標是否出現有效突破訊號
    buy_points = stock_data[
        (stock_data['Volume_Oscillator'] > 5) &                              # 當日 VO 大於 5（濾除雜訊）
        (stock_data['Volume_Oscillator'].shift(1) <= 0) &                   # 前一天小於等於 0（突破條件）
        (stock_data['MA_short'] > stock_data['MA_short'].shift(1)) &       # 均線上揚（確認價穩）
        (stock_data['VO_Positive_Count'] >= 2)                              # 近三日至少兩日有效放量
    ]

    # 創建圖表
    fig = subplots.make_subplots(rows=3, cols=1, 
                        subplot_titles=('股價與移動平均線'#, 'KD 指標'
                                        , 'MACD 指標(有價)','交易量(有量)', '成交金額與成交量震盪指標'),
                        shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        specs=[[{"secondary_y": True}],[{"secondary_y": True}] ,
                               [{"secondary_y": True}]],
                        row_heights=[0.6, 0.2, 0.2]  # 调整各行的高度比例
                        )
    ###############################################################################################
    # 添加股票箱型圖到第一圖
    fig.add_trace(go.Candlestick(x=stock_data['年月日'],
                             open=stock_data['開盤價'],
                             high=stock_data['最高價'],
                             low=stock_data['最低價'],
                             close=stock_data['收盤價'],
                             name='箱型圖',visible='legendonly',
                             increasing_line_color='red', 
                             decreasing_line_color='green',
                             increasing_fillcolor='rgba(255,0,0,0.3)',
                             decreasing_fillcolor='rgba(0,255,0,0.3)'), row=1, col=1)

    # 添加股價、短期和長期移動平均線、黃金交叉和死亡交叉到第一圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['收盤價'],                  
                             mode='lines', line_color='grey',
                             text=stock_data['Full_Summary'],name='股價'), row=1, col=1)# Trend
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_short'],                  
                             mode='lines', line_color='#ebbd67', 
                             name='短期移動平均線 (MA 5) MA_short'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_long'],                  
                             mode='lines', line_color='blue', 
                             name='長期移動平均線 (MA 20) MA_long'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_longlong'],                  
                             mode='lines', line_color='red', 
                             name='長期移動平均線 (MA 50) MA_longlong'), row=1, col=1)
        
    
    fig.add_trace(go.Scatter(x=stock_data[stock_data['MA_break']]['年月日'],
                         y=stock_data[stock_data['MA_break']]['MA_long'],
                         mode='markers', marker_symbol="star", 
                         marker_color="red", marker_size=10,
                         name='MA_break',visible='legendonly'), row=1, col=1, secondary_y=False)
    
 
    # 添加成交金額、短期和長期移動平均線、成交量震盪指標以及買入點到第二圖
    fig.add_trace(go.Bar(x=stock_data['年月日'],
                         y=stock_data['成交金額'],
                         name='成交金額',
                         marker_color=stock_data['Bar_Color'],
                         opacity=0.5), row=1, col=1, secondary_y=True)
    
    
    fig.add_trace(go.Scatter(x=stock_data[stock_data['VPC_break']]['年月日'],
                             y=stock_data[stock_data['VPC_break']]['收盤價'],
                             mode='markers', marker_symbol="star",  line_color='gold',marker_size=20,
                             name='中'), row=1, col=1)     
  
    i_row=1
    # 迴圈遍歷每個區間，將它們加入到圖形中
    for interval in merged_intervals:
        x0, x1 = interval
        fig.add_vrect(x0=x0, x1=x1,
                      annotation_text="--", annotation_position="top left",
                      fillcolor="#f6b26b", opacity=0.25, line_width=0, row=i_row, col=1)
    ###############################################################################################
    i_row=2
    ###################    添加 MACD 指標、Signal Line、DIF 以及 MACD 的黃金交叉和死亡交叉到第四圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MACD'],
                             mode='lines', line_color='#ebbd67',
                             name='Diff(12,26)'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MACD-SL'],
                             mode='lines', line_color='#67ceeb',
                             name='MACD(9)'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=macd_golden_crosses['年月日'], 
                             y=macd_golden_crosses['MACD'], 
                             mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=10,
                             name='MACD 黃金交叉'), row=i_row, col=1)
    fig.add_trace(go.Scatter(x=macd_death_crosses['年月日'], 
                             y=macd_death_crosses['MACD'], 
                             mode='markers', marker_symbol="triangle-down", marker_color="black", marker_size=10,
                             name='MACD 死亡交叉'), row=i_row, col=1)
    

    fig.add_trace(go.Scatter(x=stock_data[stock_data['macd_golden_crosses_area']]['年月日'],
                             y=stock_data[stock_data['macd_golden_crosses_area']]['MACD'],
                             mode='markers', line_color='red',
                             name='Macd上升'), row=i_row, col=1)
    

    fig.update_yaxes(title_text="MACD", row=i_row, col=1)
    
    ###############################################################################################
    i_row=3
    ###################    
    # === 主 Y 軸：VPC 動能分析 ===
    fig.add_trace(go.Scatter(x=stock_data['年月日'],  y=stock_data['VPC_MACD'], 
                             visible='legendonly', mode='lines', 
                             line_color='#ebbd67', name='VPC_MACD'), 
                  row=i_row, col=1)

    fig.add_trace(go.Scatter(x=stock_data['年月日'],  y=stock_data['VPC_SIGNAL'],  
                             visible='legendonly', mode='lines', 
                             line_color='#67ceeb', name='VPC_SIGNAL'), 
                  row=i_row, col=1)

    fig.add_trace(go.Scatter(x=stock_data['年月日'], y=stock_data['volume_threshold'], 
                             visible='legendonly', mode='lines', 
                             line_color='gray', name='VPC 動能門檻'), 
                  row=i_row, col=1)

    fig.add_trace(go.Scatter(x=stock_data[stock_data['Volume_Price_Change_break']]['年月日'],
                             y=stock_data[stock_data['Volume_Price_Change_break']]['VPC_SIGNAL'],
                             visible='legendonly', mode='markers', 
                             marker_symbol="star", marker_color="red", marker_size=10, 
                             name='Volume_Price_Change_break'), 
                  row=i_row, col=1)

    fig.update_yaxes(title_text="交易量", row=i_row, col=1)

    # === 次 Y 軸：成交量移動平均與震盪 ===
    fig.add_trace(go.Scatter(x=stock_data['年月日'], y=stock_data['Volume_Oscillator'],
                            visible='legendonly', mode='lines', 
                             line_color='gray', name='成交量震盪指標 (VO)'), 
                  row=i_row, col=1, secondary_y=True)

    fig.add_trace(go.Scatter(x=buy_points['年月日'], y=buy_points['Volume_MA_long'],
                              mode='markers', 
                             marker_symbol="triangle-up", marker_color="red", marker_size=10,
                             name='成交量增加'), 
                  row=i_row, col=1, secondary_y=True)

    fig.add_trace(go.Scatter(x=stock_data[stock_data['VPC_break']]['年月日'],
                             y=stock_data[stock_data['VPC_break']]['Volume_MA_long'],
                             mode='markers', marker_symbol="star", 
                             line_color='yellow', marker_size=20,
                             name='中'), 
                  row=i_row, col=1, secondary_y=True)

    fig.add_trace(go.Scatter(x=stock_data['年月日'], y=stock_data['Volume_MA_short'],
                             mode='lines', 
                             line=dict(width=1, dash='dot', color='orange'),
                             name='成交量MA5'), 
                  row=i_row, col=1, secondary_y=True)

    fig.add_trace(go.Scatter(x=stock_data['年月日'], y=stock_data['Volume_MA_long'],
                             mode='lines', 
                             line=dict(width=1, dash='dash', color='blue'),
                             name='成交量MA10'), 
                  row=i_row, col=1, secondary_y=True)

    # 補上次 Y 軸標題（如需要）
    fig.update_yaxes(title_text="成交量指標", row=i_row, col=1, secondary_y=True)

   ###############################################################################################  
    

    # 設置 x 和 y 軸 label
    fig.update_yaxes(title_text="股價", row=1, col=1)
    fig.update_xaxes(title_text="日期", row=4, col=1)
    
    #fig.update_yaxes(title_text="KD 指標 (%)", row=3, col=1)
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)

    ########################
    #  想在圖片右下角備註一些指標情況
    ########################
    
    # 定义灯号和文字的位置、颜色
    transparent_color = 'gray'
    indicator_labels = [
        'MACD 黃金交叉',
        'KD 黃金交叉',
        '觀察點-連續三日上升短期強勢信號',
        '轉強點-加權市場信號'
    ]
    indicator_colors = [
        'red' if stock_data['KD_golden_cross'].iloc[-1] else transparent_color,
        #'red' if stock_data['Signal_Balance_uptrend_3days'].iloc[-1] else transparent_color,
        #'red' if stock_data['Weighted_Signa_over_threshold'].iloc[-1] else transparent_color
    ]

    # 生成 HTML 内容
    html_content = ""
    for label, color in zip(indicator_labels, indicator_colors):
        html_content += f"""
        <div style="display: flex; align-items: center;">
            <div style="width: 10px; height: 10px; border-radius: 50%; background-color: {color}; margin-right: 8px;"></div>
            <span>{label}</span>
        </div><br>
    """
        
    html_iframe = f"""
    <div style="width: 100%; height: 250%; overflow: auto;">
      
        <div id="iframe-container" style="height:750px;display: none;">
            <iframe src="https://www.wantgoo.com/stock/{stock_number}" width="100%" height="100%" frameborder="0"></iframe>
        </div>
        <button onclick="toggleIframe()">顯示/隱藏 iframe</button>
        <a href="https://www.wantgoo.com/stock/{stock_number}"  target="_blank">玩股網</a>
        <a href=" https://tw.stock.yahoo.com/quote/{stock_number}.TW"  target="_blank">yahoo</a>
        <a href=" https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a={stock_number}"  target="_blank">moneydj</a>
    </div>
    <script>
        function toggleIframe() {{
            var iframeContainer = document.getElementById('iframe-container');
            if (iframeContainer.style.display === 'none') {{
                iframeContainer.style.display = 'block';
            }} else {{
                iframeContainer.style.display = 'none';
            }}
        }}
    </script>
    
    """

    # 添加 HTML 内容到图表
    html_text =html_iframe + '<br>'+ html_content+ '<br>'+ stock_data['Full_Summary'].iloc[-1]
    # MARK CONTENT 
    #html_text =html_iframe 
    # 创建新的文本区域并使用 CSS 定位到右下角
    text_area = f'''
    <style>
        .text-area {{
            position: fixed;
            bottom: 10px;
            right: 10px;
            background-color: white;
            padding: 10px;
            border: 1px solid black;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
            z-index: 1000;
            max-width: 300px;
            word-wrap: break-word;
        }}
    </style>
    <div class="text-area">
        {html_text}
    </div>
    '''

    save_plt_to_html(stock_number,stock_data,fig,text_area)

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import threading

In [14]:
global df_save_df_to_excel
global df_save_old_RunRealTimeStock
df_save_df_to_excel=pd.DataFrame()
df_save_old_RunRealTimeStock=pd.DataFrame()

def save_to_df(_df):
    _df['年月日'] = _df['年月日'].dt.strftime('%Y-%m-%d')
    
    # 處理資料 df1
    df1=_df
    
    # 處理資料 df2
    _stock_data=_df
    to_RunRealTimeStock_excel=_df
    to_RunRealTimeStock_excel['now_price']=_stock_data['收盤價']
    to_RunRealTimeStock_excel['change_price']=_stock_data['漲跌價差']
    to_RunRealTimeStock_excel['change_quote'] = (_stock_data['漲跌價差'].astype(float, errors='ignore') / _stock_data['開盤價'].astype(float, errors='ignore')).replace([float('inf'), -float('inf'), float('nan')], 0) * 100

    to_RunRealTimeStock_excel['change_quote'] = to_RunRealTimeStock_excel['change_quote'].astype(float).apply(lambda x: f"{x:.2f}%")
    df2=to_RunRealTimeStock_excel[['日期','stock_number','now_price','change_price','change_quote']]

    # 更新全局 DataFrame
    global df_save_df_to_excel  
    if  df_save_df_to_excel.empty:
        df_save_df_to_excel=df1
    else :
        df_save_df_to_excel = pd.concat([df_save_df_to_excel, df1], ignore_index=True)
       
    global df_save_old_RunRealTimeStock  
    if  df_save_old_RunRealTimeStock.empty:
        df_save_old_RunRealTimeStock=df2
    else :
        df_save_old_RunRealTimeStock = pd.concat([df_save_old_RunRealTimeStock, df2], ignore_index=True)

In [15]:

# 定义保存 Excel 文件的函数
def save_process_data(stock_number, stock_data):
    try:
        if stock_data.empty:
            print(f'DataFrame is empty. Skipping save.{stock_number}')
            return
        
        level, interval_type, lower_bound, upper_bound = get_interval(stock_data.iloc[-1]['收盤價'], np.array(stock_data['收盤價']))
        stock_data['stock_number'] = stock_number
        stock_data['level'] = level
        stock_data['interval_type'] = interval_type
        stock_data['lower_bound'] = lower_bound
        stock_data['upper_bound'] = upper_bound
        
        #存excel 
        _max_end_date=max(end_time_list)
        _min_end_date=min(end_time_list) 
        max_end_date = f"{int(_max_end_date.split('/')[0]) + 1911}-{_max_end_date.split('/')[1]}-{_max_end_date.split('/')[2]}"
        min_end_date=f"{int(_min_end_date.split('/')[0]) + 1911}-{_min_end_date.split('/')[1]}-{_min_end_date.split('/')[2]}"

        stock_data = stock_data[
                    (stock_data['年月日'] <= max_end_date) & 
                    (stock_data['年月日'] >= min_end_date)
        ]
        save_to_df(stock_data)
        
    except Exception as error:
        print(f'Error in save_process_data for stock {stock_number}: {error}')

# 定义主要爬虫和处理函数
def process_stock_codes(stock_number):
    try:
        ## Test
        # craw_stock_need_update=Truw
        #global _dummyData

        craw_stock_need_update=False
        RowData_df_craw_stock, His_Stock, isSuccess = craw_stock(stock_number, start_month,datetime.now().strftime("%Y-%m-%d"),craw_stock_need_update)
        _dummyData=data_process(RowData_df_craw_stock)

        #存html 
        gen_html(stock_number, _dummyData)  
        save_process_data( stock_number, _dummyData)
    
    except Exception as error:
        print(f'Error processing stock {stock_number}: {error}')

# 分別處理不同的股票列表
def process_stock_list(stock_list):
    for stock_number in stock_list:
        process_stock_codes(stock_number)

In [16]:
## 【Run】 跑數據
start_month = '2023-09-01'

start_date = datetime.strptime(start_month, "%Y-%m-%d").date()
today = datetime.today().date()

def to_roc(date_obj):
    roc_year = date_obj.year - 1911
    return f"{roc_year}/{date_obj.month:02d}/{date_obj.day:02d}"

end_time_list = []
current_date = start_date
while current_date <= today:
    end_time_list.append(to_roc(current_date))
    current_date += timedelta(days=1)

end_time_list=end_time_list[-10:]

min(end_time_list) 


'114/06/21'


process_stock_codes('9958')


process_stock_codes('1101')
process_stock_codes('1599')
process_stock_codes('2062')
process_stock_codes('2301')
process_stock_codes('2449')
process_stock_codes('2376')
process_stock_codes('2542')
process_stock_codes('3066')
process_stock_codes('4966')
process_stock_codes('6789')
process_stock_codes('3706')
process_stock_codes('2376')


# Test
df_save_df_to_excel